In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import tweepy
import json
import requests
import inspect
import re
from tqdm import tqdm
from time import time, sleep

In [2]:
# data = pd.read_csv("politicians.csv")
# print(data)

# usernames = data["Username"].tolist()
# ids = data["ids"].tolist()



In [3]:
with open("dev_keys_json.json", "r") as f:
    keys = json.load(f)
client = tweepy.Client(bearer_token=keys["Bearer_token"],
                       access_token = keys["API_key"],
                       access_token_secret = keys["API_secret_key"],
                       wait_on_rate_limit=True)


# empty_list_list = []
# for i in range(len(ids)):
#     empty_list_list.append({"ids":[], "usernames":[]})

# data["Following"] = empty_list_list

In [4]:
from ast import literal_eval

data = pd.read_csv("politicians_connections_two.csv")
print(data)
data["Following"] = data["Following"].apply(literal_eval)

print(data)

usernames = data["Username"].tolist()
ids = data["ids"].tolist()

          Party                      Name  Branch        Username  \
0    Republican             Pete Sessions   House    PeteSessions   
1    Democratic            Debbie Dingell   House   RepDebDingell   
2    Democratic           Pramila Jayapal   House      RepJayapal   
3    Republican               Rob Wittman   House      RobWittman   
4    Democratic           Chellie Pingree   House  chelliepingree   
..          ...                       ...     ...             ...   
521  Republican               Thom Tillis  Senate   SenThomTillis   
522  Republican         Diana Harshbarger   House  RepHarshbarger   
523  Republican          Tommy Tuberville  Senate   SenTuberville   
524  Democratic  Debbie Wasserman Schultz   House    RepDWStweets   
525  Republican              Lisa McClain   House  RepLisaMcClain   

                     ids                                          Following  
0               24735461  {'ids': [1396151678, 815966620300480514, 12912...  
1             2

In [5]:
os.chdir("./scraped_tweets")
print(os.getcwd())


/home/pi/Documents/opinion_networks/scraping_early/scraped_tweets


In [7]:
import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, z):
        if isinstance(z, datetime.datetime):
            return (str(z))
        else:
            return super().default(z)


def get_tweets(client, id, retweet = False):
    tweets = []

    if retweet:
    
        for follow in tweepy.Paginator(client.get_users_tweets, id = id, 
                                       tweet_fields=["created_at"],
                                      max_results=1000).flatten(limit = 99999):
            tweets.append(follow)
    else:
        for follow in tweepy.Paginator(client.get_users_tweets, id = id, exclude = "retweets",
                                       tweet_fields=["created_at"],
                                      max_results=100).flatten(limit = 99999):
            tweets.append(follow)        
    tweet_ids = []
    for i, t in enumerate(tweets):
        tweet_ids.append(t["id"])
    
    return tweets, tweet_ids


pbar = tqdm(ids)


#i = 0
for i, id in enumerate(pbar):
    pbar.set_description(f"{usernames[i]}")

    # if data["Following"].tolist()[i]["ids"] != []:
    #     #print(i)
    #     #i += 1
    #     continue
    
    # try:
    tweets, tweet_ids = get_tweets(client, id)
    # except TweepError as tweeperror:
    #     pbar.set_description(f'Tweep error: {tweeperror}, sleeping for 60s')
    #     sleep(60)
    #     tweets, tweet_ids = get_tweets(client, id)
    # except HTTPException as tweeperror:
    #     pbar.set_description(f'Tweep error: {tweeperror}, sleeping for 60s')
    #     sleep(60)
    #     tweets, tweet_ids = get_tweets(client, id)
    
    # print(tweets[0]["created_at"])
    tweet_dict = {"ids":tweet_ids}
    filename = f"{usernames[i]}.json"
    
    for tweet in tweets:
        tweet_dict[tweet["id"]] = {"text":tweet["text"], "time":tweet["created_at"]}
    
    # print(tweet_dict)
    
    with open(filename, "w") as outfile:
        json.dump(tweet_dict, outfile, cls=DateTimeEncoder)
    
    
    i += 1

RepLisaMcClain: 100%|███████████████████████| 526/526 [1:53:34<00:00, 12.96s/it]
